In [1]:
# This app is used to automatically generate and send to cell numbers text 
# messages that solicit leads for my real estate business
#
# How it does it: Big Picture...
# read a .csv lead infortmation file, aka "Arch"
# clean and parse Arch
# for each line of lead data...
#   read a line of lead data   
#   if lead is somehow disqualified, next line of lead data
#   else # lead was NOT disqualified so proceed...
#      construct customized text string message
#      load the text string 
#      pass the text string & cell # to Twilio which sends the message
#      verify success of message send
#      log success/failure
#      next line of lead data
#
# @Gitignore: 
# --- twilio-config
# --- All Arch lead source files except testmm?
# --- Others?
#
# Improvements in the queue...
# 1. Ignore the cell number if its a DNC 
#        --> DONE.
# 2. Add command line parameter extraction for...
#     A. to define the .xls source file name 
#         --> DONE via reading the /1csvfiles folder for source data
#     B. to set debug mode: does everything EXCEPT send the text to Twilio
#         --> DONE via the aalgx_config: Enable_SendText = False
# 3. ???
#
# Other:
# There are to be two versions of this app...
#   1. the Jupyter notebook version, to be run from JN for design & debug
#   -- read the name of the Arch source file to be used from inline source = not ideal
#   2. for productio, the command line version, to be run from shell or VsCode
#   -- read the name of the Arch source file to be used from the command line
#   -- this is closer to production; maybe cretae a windows app someday?
#   -- see rempest_cmd.py from machine learning final project for good sample code


import pandas as pd
# import namelist.py for when I create a dictionary of names
from twilio.rest import Client
from twilio_config import account_sid, auth_token, from_
from aalgx_config import Enable_SendText, ref_url, my_intro, trec_comply, action

In [2]:
def getFirstName(line_num):
    orig_name_str = []
    final_parsed_name = ["#","#","#","#","#"]
    temp_name_1 = []
    temp_name_2 = []
    last_name = []
    first_name_primary = []
    first_name_secondary = []
    suggested_name_1st = ''
    #get the contact firstname miscname lastname string
    orig_name_str = new_axe_pd.loc[line_num,"Contact"]
    print('Contact string read: "',orig_name_str,'"...',end="")
    #If string contains a "," the format is probably "lastname,firstname"
    print("final_parsed_name BEFORE parsing:",final_parsed_name)
    if ("," in orig_name_str):
        # first split on "," and ignore lastname
        last_name = orig_name_str.split(",") 
        print("COMMA separated last_name, first name [list]: ",last_name)
        
        if (" " in last_name[1]):
            temp_name_1 = last_name[1].split(" ")
            
            if ("&" in temp_name_1):
                temp_name_2 = temp_name_1[1].split("&")
            
                for i in range(len(temp_name_2)):
                    final_parsed_name[i+1] = temp_name_2[i]
            
            final_parsed_name[0] = temp_name_1[0]
        else:
            final_parsed_name[0] = last_name[1] ### kind of UGLY, but it works. Post MVP Refactor?
        
    else:
        final_parsed_name = orig_name_str.split(" ")
        #
        # Patterns: If here and both names are CAPITALIZED, the 2nd entry is probably the FIRST name!!
        if len(final_parsed_name)>1 and (final_parsed_name[0].isupper() and final_parsed_name[1].isupper()):
            final_parsed_name.append("#")
            final_parsed_name.append("#")
            final_parsed_name.append("#")
            final_parsed_name[4] = final_parsed_name[0] #save last name at end
            final_parsed_name[0] = final_parsed_name[1]
            final_parsed_name[1] = "#"
        #Handle "Familyname Living Trust" case
        if ("Trust" in final_parsed_name) or ("Living" in final_parsed_name):
            final_parsed_name[0] = final_parsed_name[0]+" Family"
            final_parsed_name.append("#")
        # 
        print("ONLY SPACES: final_parsed_name AFTER parsing:",final_parsed_name)
        # potenitally other names in [2:3] ??
    suggested_name_1st = final_parsed_name[0]
    print(" Parsed as:",final_parsed_name)
    print("To accept '",suggested_name_1st,"' as first, hit 'Y' else enter [index] of replacement OR NameString: ")
    return suggested_name_1st

    def getUserApproval():
        selection = input()
        # selection = "Y" # these two  lines let it run free w/o operator
        if len(selection) > 1:
            confirm = input(f'You entered {selection} to be the first name. Are you sure? Y/N ')
            if confirm == 'Y' or confirm == 'y':
                return selection
            else:
                return "error"
        elif selection == 'Y' or selection == 'y':
            return suggested_name_1st
        elif selection.isdigit(): # if index selected, replace the first name with the slected name
            return final_parsed_name[int(selection)]
        else: return "error"

In [3]:
def GetCellNumbers():    
#     Index(['HomeZip', 'Price', 'Type', 'Address', 'Zip', 'Location', 'Contact',
#        'p1', 'p1dnc', 'p1cell', 
#        'p2', 'p2dnc', 'p2cell', 
#        'p3', 'p3dnc', 'p3cell', 
#        'MatchedPhone'],
        
    ph1 = str(new_axe_pd.loc[lg,'p1'])[0:10]
    nc1 = new_axe_pd.loc[lg,'p1dnc']
    ic1 = new_axe_pd.loc[lg,'p1cell']

    ph2 = str(new_axe_pd.loc[lg,'p2'])[0:10]
    nc2 = new_axe_pd.loc[lg,'p2dnc']
    ic2 = new_axe_pd.loc[lg,'p2cell']

    ph3 = str(new_axe_pd.loc[lg,'p3'])[0:10]
    nc3 = new_axe_pd.loc[lg,'p3dnc']
    ic3 = new_axe_pd.loc[lg,'p3cell']
    
#     print("Phone # list & cell flags: ","  ph1: ",ph1,"  ic1: ",ic1,\
#                                         "  ph2: ",ph2,"  ic2: ",ic2,\
#                                         "  ph3: ",ph3,"  ic3: ",ic3)

    if len(ph1)==10 and nc1 == '?' and ic1=='Y': cellnumbers.append(ph1)
    if len(ph2)==10 and nc2 == '?' and ic2=='Y': cellnumbers.append(ph2)
    if len(ph3)==10 and nc3 == '?' and ic3=='Y': cellnumbers.append(ph3)

#    print("cellnumbers B4 duplicate check: ", cellnumbers)
#   here we handle the not uncommon case when there are duplicate cell numbers
    if len(cellnumbers) > 2:
        if cellnumbers[0]==cellnumbers[1] or cellnumbers[0]==cellnumbers[2]: del cellnumbers[0]
        elif cellnumbers[1]==cellnumbers[2]: del cellnumbers[1]
    if len(cellnumbers) > 1:
        if cellnumbers[0]==cellnumbers[1]: del cellnumbers[0]
#    print("cellnumbers AFTER duplicate check: ", cellnumbers)

    #convert to a string
    # test the result

In [4]:
# TWILIO TEXT MESSAGE SENDER
# https://www.twilio.com/
def SendText(pbody,ptextdest):
# ptextdest is constant to Micheman cell until full test is complete
    #
    # parameters passed...
    # pbody       = body text string parameter
    # ptextdest   = the text msg destination (phone number) for the lead I am targeting
    
     client = Client(account_sid, auth_token)
     message = client.messages.create(body=pbody,
                                      from_='+19403103003',
                                      to= f'[+][1][{ptextdest}]')
     return(message.sid)

    
#     # Twilio sample code says: "Get your Account Sid and Auth Token from twilio.com/console"
#     client = Client(account_sid, auth_token)
#     message = client.messages.create(body=pbody, to= f'[+][1][{ptextdest}]')
#     return(message.sid)

In [5]:
def ComposeTextMsg(listing_index,name):
    line_1 = 'Hello ' + name + my_intro
    listing_Address = new_axe_pd.loc[listing_index, "Address"]
    line_2 = "I noticed your property at " + listing_Address + " is no longer listed on the MLS.\n"
    # line_3 = "If you still want to sell, click on my web link: " + ref_url + " to see why I can help.\n"
    line_3 = "I will reach out sometime early next week with a live call\n"
    line_4 = "Already relisted or sold? Please disregard this message.\n"
    line_5 = trec_comply
    return line_1+line_2+line_3+line_4+line_5

In [6]:
# "lead_source_file_name" is defined in aalgx-config.py
import os, fnmatch
folder_name_csv_source = './1csvfiles/' # where the .csv source data files live
folder_name_rsl_files  = './2rslfiles/' # where the result data files live (actual generated data)
folder_name_log_files  = './3logfiles/' # where the log data files live (summaries of results)
listOfFiles = os.listdir(folder_name_csv_source)
print(listOfFiles)
pattern = "*.csv"
number_matches = 0
for entry in listOfFiles: # we only just grab the first entry for now and ignore any others
    if fnmatch.fnmatch(entry, pattern):
        number_matches += 1
        full_name = folder_name_csv_source + entry
        full_entry_str = entry.split(".")
        file_name = full_entry_str[0] # need raw filename w/o ext for log & rsl files later
        print(f'Data file to be used for this run: {full_name}')
        pd.set_option('display.max_columns', 999) # hide no columns in pandas display
        pd_axe = pd.read_csv(full_name) # July 1st: lots of ones
        break # we only just grab first entry for now and ignore any others
if number_matches < 1:
    print("Error detected: No 'filename.csv' source data file found, abort this run !")

['AFX191019.csv', 'misc_csv_files', 'oldn_csv_files', 'used_csv_files']
Data file to be used for this run: ./1csvfiles/AFX191019.csv


In [7]:
pd_axe

,HomeZip,Date,ListingID,Miles,Price,Source,Type,Address,Zip,Location,Contact,Phone1,PhoneNo1IsDNC,PhoneNo1IsCell,Phone2,PhoneNo2IsDNC,PhoneNo2IsCell,Phone3,PhoneNo3IsDNC,PhoneNo3IsCell,EnhancedName,EnhancedAddress,EnhancedCity,EnhancedState,EnhanedZip,MatchedPhone,MatchedName,MatchedCity,MatchedState,MatchedZip
0,76226,10/19/2019 6:31:12 AM,11810250,7,385000,EXPIRED,EXPIRED,220 Saratoga Drive,75065,Steeplechase North Add PhHickory Creek,Julia Schott,8155469715,Y,Y,8.152589e+09,NaN,Y,4.699935e+09,NaN,NaN,Julia Schott,220 Saratoga Dr,Hickory Creek,TX,75065-0330,815-546-9715,Schott Julia Lynn:,NaN,NaN,NaN
1,76226,10/19/2019 6:31:13 AM,11810257,9,499999,EXPIRED,EXPIRED,4541 Seventeen Lakes Court,76262,Seventeen Lakes AddFort Worth,Gary R Gibson,8173077968,NaN,Y,8.179072e+09,Y,Y,8.179072e+09,Y,Y,Gary R Gibson,4541 Seventeen Lakes Ct,Roanoke,TX,76262-3741,817-307-7968,Gibson Gary Ray:,NaN,NaN,NaN
2,76226,10/19/2019 6:31:16 AM,11810270,11,950000,EXPIRED,EXPIRED,705 Silver Spur Court,76092,Clariden Ranch Ph IbSouthlake,Tienhsiang E Wang,8174909810,NaN,NaN,NaN,NaN,NaN,8.174910e+09,NaN,NaN,Tienhsiang E Wang,1818 High Country Dr,Westlake,TX,76262-4813,817-490-9810,Wang Ming Yuan:,NaN,NaN,NaN
3,76226,10/19/2019 6:31:04 AM,11810215,14,249000,EXPIRED,EXPIRED,2129 Woodhaven Drive,75068,Lakewood Estates Ph 2Little Elm,Pedro Serrano,9724523696,Y,NaN,9.722077e+09,Y,Y,9.722077e+09,Y,Y,Pedro Serrano,2129 Woodhaven Dr,Little Elm,TX,75068-5119,972-452-3696,Serrano Maria Guadalupe:,NaN,NaN,NaN
4,76226,10/19/2019 6:31:07 AM,11810227,14,310000,EXPIRED,EXPIRED,712 Marietta Lane,76227,Marietta Village At Savannah PAubrey,Mario A Figueroa,9725229802,Y,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Mario A Figueroa,712 Marietta Ln,Savannah,TX,76227-1252,972-522-9802,Figueroa Mario A:,NaN,NaN,NaN
5,76226,10/19/2019 6:31:08 AM,11810233,14,329900,EXPIRED,EXPIRED,741 Sparrow Lane,75019,Northlake Woodlands East Ph 02Coppell,Hugo J Bonacci,3256750806,NaN,NaN,6.828885e+09,NaN,NaN,3.256656e+09,Y,Y,Hugo J Bonacci,741 Sparrow Ln,Coppell,TX,75019-3458,325-675-0806,Bonacci Mary Denise:,NaN,NaN,NaN
6,76226,10/19/2019 6:31:09 AM,11810240,15,349900,EXPIRED,EXPIRED,10229 Vintage Drive,76244,Crawford Farms AddFort Worth,John R Emmerich,8178880041,Y,Y,NaN,NaN,NaN,8.179641e+09,Y,Y,John R Emmerich,10229 Vintage Dr,Fort Worth,TX,76244-6653,817-888-0041,Emmerich Chad William:,NaN,NaN,NaN
7,76226,10/19/2019 6:31:15 AM,11810267,15,729900,EXPIRED,EXPIRED,4102 Savannah Court,76034,Tara Plantation AddColleyville,Alesha N Foreman,8173071637,Y,Y,NaN,NaN,NaN,8.175015e+09,Y,Y,Alesha N Foreman,4102 Savannah Ct,Colleyville,TX,76034-4108,817-307-1637,Foreman Alesha Nicole:,NaN,NaN,NaN
8,76226,10/19/2019 6:31:16 AM,11810273,15,1195000,EXPIRED,EXPIRED,4904 Rockrimmon Court,76034,RockrimmonColleyville,Tony Bryson,8177939753,Y,Y,6.827388e+09,NaN,NaN,8.012016e+09,Y,Y,Tony Bryson,4904 Rockrimmon Ct,Colleyville,TX,76034-3544,817-793-9753,Bryson Kathleen Marie:,NaN,NaN,NaN
9,76226,10/19/2019 6:31:06 AM,11810222,16,289900,EXPIRED,EXPIRED,3012 Waterfall Drive,76177,Valley RidgeFort Worth,Eric M Wuebben,9492746476,Y,NaN,NaN,NaN,NaN,6.264455e+09,Y,NaN,Eric M Wuebben,3012 Waterfall Dr,Fort Worth,TX,76177-2600,949-274-6476,Wuebben Eric M:,NaN,NaN,NaN


In [8]:
pd_axe.rename(columns={'Phone1':'p1','PhoneNo1IsDNC':'p1dnc','PhoneNo1IsCell':'p1cell',
                       'Phone2':'p2','PhoneNo2IsDNC':'p2dnc','PhoneNo2IsCell':'p2cell',
                       'Phone3':'p3','PhoneNo3IsDNC':'p3dnc','PhoneNo3IsCell':'p3cell',},
              inplace=True)

In [9]:
pd_axe_dc = pd_axe.drop(["Date","ListingID","Miles","Source",
                         "MatchedCity","MatchedState","MatchedZip", \
                    #    'p1dnc','p2dnc','p3dnc',\
                         'EnhancedName', 'EnhancedAddress','EnhancedCity', 'EnhancedState', 'EnhanedZip',
                         "MatchedName","MatchedCity","MatchedState","MatchedZip"],
                         axis=1)

In [10]:
pd_axe_dc.columns

Index(['HomeZip', 'Price', 'Type', 'Address', 'Zip', 'Location', 'Contact',
       'p1', 'p1dnc', 'p1cell', 'p2', 'p2dnc', 'p2cell', 'p3', 'p3dnc',
       'p3cell', 'MatchedPhone'],
      dtype='object')

In [11]:
pd_axe_clean = pd_axe_dc.fillna("?")

In [12]:
pd_axe_clean

,HomeZip,Price,Type,Address,Zip,Location,Contact,p1,p1dnc,p1cell,p2,p2dnc,p2cell,p3,p3dnc,p3cell,MatchedPhone
0,76226,385000,EXPIRED,220 Saratoga Drive,75065,Steeplechase North Add PhHickory Creek,Julia Schott,8155469715,Y,Y,8.15259e+09,?,Y,4.69994e+09,?,?,815-546-9715
1,76226,499999,EXPIRED,4541 Seventeen Lakes Court,76262,Seventeen Lakes AddFort Worth,Gary R Gibson,8173077968,?,Y,8.17907e+09,Y,Y,8.17907e+09,Y,Y,817-307-7968
2,76226,950000,EXPIRED,705 Silver Spur Court,76092,Clariden Ranch Ph IbSouthlake,Tienhsiang E Wang,8174909810,?,?,?,?,?,8.17491e+09,?,?,817-490-9810
3,76226,249000,EXPIRED,2129 Woodhaven Drive,75068,Lakewood Estates Ph 2Little Elm,Pedro Serrano,9724523696,Y,?,9.72208e+09,Y,Y,9.72208e+09,Y,Y,972-452-3696
4,76226,310000,EXPIRED,712 Marietta Lane,76227,Marietta Village At Savannah PAubrey,Mario A Figueroa,9725229802,Y,?,?,?,?,?,?,?,972-522-9802
5,76226,329900,EXPIRED,741 Sparrow Lane,75019,Northlake Woodlands East Ph 02Coppell,Hugo J Bonacci,3256750806,?,?,6.82888e+09,?,?,3.25666e+09,Y,Y,325-675-0806
6,76226,349900,EXPIRED,10229 Vintage Drive,76244,Crawford Farms AddFort Worth,John R Emmerich,8178880041,Y,Y,?,?,?,8.17964e+09,Y,Y,817-888-0041
7,76226,729900,EXPIRED,4102 Savannah Court,76034,Tara Plantation AddColleyville,Alesha N Foreman,8173071637,Y,Y,?,?,?,8.17501e+09,Y,Y,817-307-1637
8,76226,1195000,EXPIRED,4904 Rockrimmon Court,76034,RockrimmonColleyville,Tony Bryson,8177939753,Y,Y,6.82739e+09,?,?,8.01202e+09,Y,Y,817-793-9753
9,76226,289900,EXPIRED,3012 Waterfall Drive,76177,Valley RidgeFort Worth,Eric M Wuebben,9492746476,Y,?,?,?,?,6.26445e+09,Y,?,949-274-6476


In [13]:
new_axe_pd = pd_axe_clean
new_axe_pd
# Is "matched_phone" of any value to us ???'
# Is it a duplicate of any of the Ph1-3 or a different/new number matched to any related address?

,HomeZip,Price,Type,Address,Zip,Location,Contact,p1,p1dnc,p1cell,p2,p2dnc,p2cell,p3,p3dnc,p3cell,MatchedPhone
0,76226,385000,EXPIRED,220 Saratoga Drive,75065,Steeplechase North Add PhHickory Creek,Julia Schott,8155469715,Y,Y,8.15259e+09,?,Y,4.69994e+09,?,?,815-546-9715
1,76226,499999,EXPIRED,4541 Seventeen Lakes Court,76262,Seventeen Lakes AddFort Worth,Gary R Gibson,8173077968,?,Y,8.17907e+09,Y,Y,8.17907e+09,Y,Y,817-307-7968
2,76226,950000,EXPIRED,705 Silver Spur Court,76092,Clariden Ranch Ph IbSouthlake,Tienhsiang E Wang,8174909810,?,?,?,?,?,8.17491e+09,?,?,817-490-9810
3,76226,249000,EXPIRED,2129 Woodhaven Drive,75068,Lakewood Estates Ph 2Little Elm,Pedro Serrano,9724523696,Y,?,9.72208e+09,Y,Y,9.72208e+09,Y,Y,972-452-3696
4,76226,310000,EXPIRED,712 Marietta Lane,76227,Marietta Village At Savannah PAubrey,Mario A Figueroa,9725229802,Y,?,?,?,?,?,?,?,972-522-9802
5,76226,329900,EXPIRED,741 Sparrow Lane,75019,Northlake Woodlands East Ph 02Coppell,Hugo J Bonacci,3256750806,?,?,6.82888e+09,?,?,3.25666e+09,Y,Y,325-675-0806
6,76226,349900,EXPIRED,10229 Vintage Drive,76244,Crawford Farms AddFort Worth,John R Emmerich,8178880041,Y,Y,?,?,?,8.17964e+09,Y,Y,817-888-0041
7,76226,729900,EXPIRED,4102 Savannah Court,76034,Tara Plantation AddColleyville,Alesha N Foreman,8173071637,Y,Y,?,?,?,8.17501e+09,Y,Y,817-307-1637
8,76226,1195000,EXPIRED,4904 Rockrimmon Court,76034,RockrimmonColleyville,Tony Bryson,8177939753,Y,Y,6.82739e+09,?,?,8.01202e+09,Y,Y,817-793-9753
9,76226,289900,EXPIRED,3012 Waterfall Drive,76177,Valley RidgeFort Worth,Eric M Wuebben,9492746476,Y,?,?,?,?,6.26445e+09,Y,?,949-274-6476


In [14]:
new_axe_pd.columns

Index(['HomeZip', 'Price', 'Type', 'Address', 'Zip', 'Location', 'Contact',
       'p1', 'p1dnc', 'p1cell', 'p2', 'p2dnc', 'p2cell', 'p3', 'p3dnc',
       'p3cell', 'MatchedPhone'],
      dtype='object')

In [15]:
# lg = 5 # for listing lg in listings loop will get set up here...
listing_seq_id = 0
num_lg_no_text = 0
num_lg_text_sent = 0
print(f"folder_name_rsl_files: {folder_name_rsl_files}")
print(f"file_name: {file_name}")
rsl_file_name = folder_name_rsl_files+file_name+"_rsl.txt"
with open(rsl_file_name,"w") as outfile:
    for lg in range(0,len(new_axe_pd)):
        # print("lg: ",lg)
        listing_seq_id += 1
        flsid = f'\n{listing_seq_id}: '
        outfile.write(flsid)
        if (new_axe_pd.loc[lg,'Type'] == 'EXPIRED'): # Ignore listing if anything but EXPIRED (ex: FSBO)
            cellnumbers = []
            GetCellNumbers()
            price_limit = 500000
            price = int(new_axe_pd.iloc[lg,1])
            if (len(cellnumbers)>0 and price <= price_limit):
                first_name = getFirstName(lg).title() #
                if (first_name != "Error" and len(first_name) > 1):
                    text_2_name = f'  First name selected: {first_name}\n'
                    outfile.write(text_2_name)
                    # pd_axe_row_Contact = new_axe_pd.loc[lg, "Contact"]  <<------redundnant? rleft over previosu revision???
                    text2send = ComposeTextMsg(lg,first_name)
                    for cellnumber in cellnumbers:
                        text_msg_is = "@"+cellnumber+": "+text2send
                        print(text_msg_is)
                        outfile.write(text_msg_is)
                        if Enable_SendText == True:
                            texter_rtn_val = SendText(text2send,cellnumber)
    #                        texter_rtn_val = SendText(text2send,"9729980900")
                            if len(texter_rtn_val) == 34:
                                text_success = "Text Success? "+texter_rtn_val+"\n"
                                outfile.write(text_success)
                                num_lg_text_sent += 1
                            else:
                                text_fail = "Text Failure? "+texter_rtn_val+"\n"
                                outfile.write(text_fail)
                        else:
                            outfile.write("Enable_SendText == False, No Twilio was sent.\n")
                        # end cellnumbers loop
                else:
                    num_lg_no_text += 1
                    name_error = f'  Name selection or length error detected, skipping listing # {lg}\n'
                    outfile.write(name_error)
                    print(name_error)
            else:
                # reject on price or lack of cell numbers
                num_lg_no_text += 1
                if price > price_limit:
                    price_exceeded = f'  Listing Sequence ID #{listing_seq_id}: price of {price} exceeded limit. No text sent\n'
                    outfile.write(price_exceeded)                    
                if (len(cellnumbers)==0):
                    no_cell_no_text = f'  Listing Sequence ID #{listing_seq_id}: listing has no cell #, No text sent\n'
                    outfile.write(no_cell_no_text)
        else:
            type_not_exp = f'  Type detected is not EXPIRED, listing {lg} skipped.\n'
            outfile.write(type_not_exp)
            num_lg_no_text += 1
    # print("Total listings processed: ",listing_seq_id)
    
    
    end_list_msg = f'\nProcessing ended at listing sequence ID: {listing_seq_id}\n'
    num_no_text_sent = f'Total listings with no text sent: {num_lg_no_text}\n'
    num_total_text_sent = f'Total text messages sent: {num_lg_text_sent}\n'

    outfile.write(end_list_msg)
    outfile.write(num_no_text_sent)
    outfile.write(num_total_text_sent)

    print(end_list_msg)
    print(num_no_text_sent)
    print(num_total_text_sent)
rsl_file_name = folder_name_rsl_files+file_name+"_rsl.txt"
logfile_path = folder_name_rsl_files+"aalgx_logfile.txt"
with open(logfile_path,"a") as stats_file:
    head_line = "Texts stats for listing file " + f'{full_name}\n'
    stats_file.write(head_line)
    stats_file.write(end_list_msg)
    stats_file.write(num_no_text_sent)
    stats_file.write(num_total_text_sent)
    stats_file.write("\n")
print("Done.")

#
#Error to fix: why shoudd Twilio blow up my program instead of just an error return and be doen with it??
# TwilioRestException: HTTP 400 error: Unable to create record: The message From/To pair violates a blacklist rule.
# 


folder_name_rsl_files: ./2rslfiles/
file_name: AFX191019
Contact string read: " Julia Schott "...final_parsed_name BEFORE parsing: ['#', '#', '#', '#', '#']
ONLY SPACES: final_parsed_name AFTER parsing: ['Julia', 'Schott']
 Parsed as: ['Julia', 'Schott']
To accept ' Julia ' as first, hit 'Y' else enter [index] of replacement OR NameString: 
@8152589039: Hello Julia
My name is Mike and I'm a local realtor.
I specialize in expired listings.
I noticed your property at 220 Saratoga Drive is no longer listed on the MLS.
I will reach out sometime early next week with a live call
Already relisted or sold? Please disregard this message.
Thanks! Mike Maniguet, Realtor with  Keller Williams Realty of Denton TX

Contact string read: " Gary R Gibson "...final_parsed_name BEFORE parsing: ['#', '#', '#', '#', '#']
ONLY SPACES: final_parsed_name AFTER parsing: ['Gary', 'R', 'Gibson']
 Parsed as: ['Gary', 'R', 'Gibson']
To accept ' Gary ' as first, hit 'Y' else enter [index] of replacement OR NameStri